In [1]:
import numpy as np
import json
import pandas as pd
import matplotlib as plt
import seaborn as sns
import csv
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream, API, Cursor
import requests
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import os
import re

In [3]:
## access token informations 
access_token1 = "1222829174802538496-AXkAccE1dWMjuEFlEYChpgOibc6SbF"
access_token_secret1 = "jtr66CvVvspsANplSBpuMEQR5iLwK2fRtzM6aDhaC1rZT"

consumer_key1 = "sL9E5QL83DVEshfttLjEEj930"
consumer_secret1 = "R9L4Ar7UWTKB4WQw7cEoybumyTInZ6pYP3cQ5GFlWYFllYAeec"

In [4]:
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

In [6]:
OUTPUT_FILENAME = "other-outputs/tweets_US_Election_2020.json"
stop_condition = 500

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["Donald", "Trump", "Joe", "Biden", "America", "USA"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)

In [7]:
json_Election_file = []
for line in open(OUTPUT_FILENAME, 'r'):
    json_Election_file.append(json.loads(line))

In [8]:
dict_train={}
i=0
for train_file in json_Election_file:
    dict_train[i]=train_file
    i+=1
train = pd.DataFrame.from_dict(dict_train, orient='index')
train.reset_index(level=0, inplace=False)
train.head(10)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,favorited,retweeted,filter_level,lang,timestamp_ms,display_text_range,extended_tweet,possibly_sensitive,extended_entities,withheld_in_countries
0,Sat Nov 28 14:26:24 +0000 2020,1332692321729699843,1332692321729699843,RT @tedcruz: It’s bizarre to see a former head...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,False,False,low,en,1606573584383,NaN,NaN,NaN,NaN,NaN
1,Sat Nov 28 14:26:24 +0000 2020,1332692321641623553,1332692321641623553,RT @TheRISEofROD: President Trump WON all 50 s...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,False,False,low,en,1606573584362,NaN,NaN,NaN,NaN,NaN
2,Sat Nov 28 14:26:24 +0000 2020,1332692321616482305,1332692321616482305,"@hazechu Anonymous, new account, 14 followers,...","<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.332687e+18,1332687475643457536,8.955955e+08,895595520,...,False,False,low,en,1606573584356,"[9, 140]","{'full_text': '@hazechu Anonymous, new account...",False,NaN,NaN
3,Sat Nov 28 14:26:24 +0000 2020,1332692322077773824,1332692322077773824,RT @KristySwansonXO: The evil lies that these ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,False,False,low,en,1606573584466,NaN,NaN,NaN,NaN,NaN
4,Sat Nov 28 14:26:24 +0000 2020,1332692322128113665,1332692322128113665,RT @johngocee: #BidenCheated2020 Saint Joe!...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,False,False,low,en,1606573584478,NaN,NaN,False,"{'media': [{'id': 1332665469615173632, 'id_str...",NaN
5,Sat Nov 28 14:26:24 +0000 2020,1332692321931046920,1332692321931046920,@ruessionnoCatoM @HipHopDX @LilTunechi He got ...,"<a href=""http://twitter.com/download/iphone"" r...",False,1.332692e+18,1332692126078001152,3.031182e+09,3031182325,...,False,False,low,en,1606573584431,"[39, 101]",NaN,NaN,NaN,NaN
6,Sat Nov 28 14:26:24 +0000 2020,1332692322056884225,1332692322056884225,@kotstot1 @JoeBiden A true and present danger ...,"<a href=""http://twitter.com/download/android"" ...",True,1.332486e+18,1332485976648716288,2.777426e+09,2777425891,...,False,False,low,en,1606573584461,"[20, 140]",{'full_text': '@kotstot1 @JoeBiden A true and ...,NaN,NaN,NaN
7,Sat Nov 28 14:26:24 +0000 2020,1332692322144940032,1332692322144940032,RT @gatewaypundit: BREAKING HUGE: Pennsylvania...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,False,False,low,en,1606573584482,NaN,NaN,NaN,NaN,NaN
8,Sat Nov 28 14:26:24 +0000 2020,1332692321981386758,1332692321981386758,RT @GeorgePapa19: Joe Biden had “80 million vo...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,False,False,low,en,1606573584443,NaN,NaN,NaN,NaN,NaN
9,Sat Nov 28 14:26:24 +0000 2020,1332692322451152896,1332692322451152896,RT @TBrno11: @realDonaldTrump That’s not the w...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,False,False,low,en,1606573584555,NaN,NaN,NaN,NaN,NaN


In [9]:
pd.set_option('display.max_columns', None)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1503 entries, 0 to 1502
Data columns (total 37 columns):
created_at                   1503 non-null object
id                           1503 non-null int64
id_str                       1503 non-null object
text                         1503 non-null object
source                       1503 non-null object
truncated                    1503 non-null bool
in_reply_to_status_id        219 non-null float64
in_reply_to_status_id_str    219 non-null object
in_reply_to_user_id          232 non-null float64
in_reply_to_user_id_str      232 non-null object
in_reply_to_screen_name      232 non-null object
user                         1503 non-null object
geo                          0 non-null object
coordinates                  0 non-null object
place                        9 non-null object
contributors                 0 non-null object
retweeted_status             1071 non-null object
quoted_status_id             409 non-null float64
quoted_stat

In [10]:
#Normalize functions:

#NO SALEN HASHTAGS NI DEL TEXT NI DE ENTITIES.MIRAR RETWEETED STATUS Y TIENE QUE DEVOLVER
def get_hashtags(data: pd.DataFrame) -> None:
    hashtag_list=[]
    for i in range(len(data)):
        hashtags=[]
        try:
            hashtags_info = data['entities']['hashtags']
        except:
            hashtags_info = data['entities'][i]['hashtags']
        for i in range (len(hashtags_info)):
            hashtags.append(hashtags_info[i]["text"])
        hashtag_list.append(hashtags)
    data['hashtags']=hashtag_list

def remove_unnecessary_columns(data: pd.DataFrame)-> pd.DataFrame:
    cols_to_remove=['id', 'display_text_range', 'in_reply_to_status_id', 
                    'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'geo', 'coordinates', 
                    'place', 'contributors', 'is_quote_status', 'quote_count','lang', 
                    'possibly_sensitive', 'quoted_status_id','quoted_status_id_str', 'quoted_status', 
                    'quoted_status_permalink']
    
    data=data.drop(columns=cols_to_remove)
    return data

#retweeted_status only exists if the tweet has been retweeted by someone. If not, extended_tweet exists if Truncated is true.
#If both are not the case, we use directly the information given in the tweet.
def get_tweet_data(data: pd.DataFrame) -> pd.DataFrame:
    #  Tweet| Username | Date | Hashtags | Likes | Retweets | Url
    tweets_text =[]
    tweets_username =[]
    tweets_date = []
    tweets_hashtags = []
    tweets_likes = []
    tweets_retweets = []
    tweets_url = []
    for tweet in range(len(data)):
        
        if str(data.iloc[tweet]['retweeted_status'])=='nan':
            if(data["truncated"][tweet] == False):
                tweets_text.append(data['text'][tweet])
                tweets_hashtags.append(get_hashtags(data))
            else:
                tweets_text.append(data['extended_tweet'][tweet]['full_text'])
                tweets_hashtags.append(get_hashtags(data['extended_tweet'][tweet]))
            
            tweets_date.append(data['created_at'][tweet])
            tweets_username.append(data['user'][tweet]['name'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
            tweets_likes.append(data['favorite_count'][tweet])
            tweets_retweets.append(data['retweet_count'][tweet])
        else:
            tweets_text.append(data['retweeted_status'][tweet]['text'])
            #We supposed it is the name and not the @name (screen_name)
            tweets_username.append(data['retweeted_status'][tweet]['user']['name']) 
            tweets_date.append(data['retweeted_status'][tweet]['created_at'])
            tweets_hashtags.append(get_hashtags((data['retweeted_status'][tweet])))
            tweets_likes.append(data['retweeted_status'][tweet]['favorite_count'])
            tweets_retweets.append(data['retweeted_status'][tweet]['retweet_count'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
        
    tweet_info = {'Tweet': tweets_text, 'Username': tweets_username, 'Date': tweets_date, 'Hashtags': tweets_hashtags, 
         'Likes': tweets_likes, 'Retweets': tweets_retweets, 'Url': tweets_url}

    return pd.DataFrame(data=tweet_info)

In [11]:
def decontracted(text: str) -> str:
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

def punctuation_removal(text: str) -> str:
    tokenizer=RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(decontracted(text.lower()))

def stemming(text: list) -> list:
    ps = PorterStemmer()
    stemmed_text = []
    for word in text: 
        stemmed_text.append(ps.stem(word))
    return stemmed_text

def remove_stop_words(word_list: list) -> list:
    stop_words = set(stopwords.words('english'))
    text_without_stopwords = []
    for word in word_list:
        if not word in stop_words:
            text_without_stopwords.append(word)
    return text_without_stopwords

def text_normalization(text: str) -> str:
    text_normalized = punctuation_removal(text)
    text_stemmed  = stemming(text_normalized)
    text_without_stopwords = remove_stop_words(text_stemmed)
    return " ".join(text_without_stopwords)

In [12]:
train=remove_unnecessary_columns(train)

In [ ]:
def inverted_index(data: pd.DataFrame)->dict:
    inv_index={}
    for doc_num in range(len(data['text'])):
        words=train['text'][doc_num].split()
        for word in words:
            if word in inv_index:
                inv_index[word].append(doc_num)
            else:
                inv_index[word]=[doc_num]
    return inv_index

def tf_idf(docs: pd.DataFrame, inv_index: dict) -> list:
    tf=[]
    df={}
    idf={}
    tf_idf=[]
    N=len(docs)
    for doc_text in range(N):
        tf_doc_i={}
        text=docs["text"][doc_text]
        words=text.split()
        for word in words:
            if word in tf_doc_i:
                tf_doc_i[word]+=1/len(words)
            else:
                tf_doc_i[word]=1/len(words)
        tf.append(tf_doc_i)
        
    for word in inv_index:
        df[word]=len(inv_index[word])
        idf[word]=np.log(float(N/df[word]))
        
    for doc_text in range(N):
        tf_idf_per_doc={}
        for word in tf[doc_text]:
            tf_idf_per_doc[word]=tf[doc_text][word]*idf[word]
        tf_idf.append(tf_idf_per_doc)
    return tf_idf

def tf_idf_normalization(tf_idf: list):
    D_squ=[np.zeros(len(tf_idf))]
    for i in range(len(tf_idf)):
        document_values=list(tf_idf[i].values())
        for j in range(len(document_values)):
            document_values[j]=(document_values[j]**2)
            D_squ[i]+=document_values[j]
    print(D_squ)

In [14]:
#def ranking_documents(query: str, tfidf: list):
    

In [15]:
database=get_tweet_data(train)
for i in range(len(train)):
    train['text'][i]=text_normalization(train['text'][i])

C:\Users\TESTER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
tfidf=tf_idf(train, inverted_index(train))
tf_idf_normalization(tfidf)

In [ ]:
print("Insert your query:\n")
query = input()
query=text_normalization(query)
#docs = search(query, index)    
top = 20

Insert your query:



In [ ]:
# get always full text by checking if truncated. If it is, then find extended_tweet or retweeted_status.

In [ ]:
os.remove(OUTPUT_FILENAME)

In [ ]:
#Quitar URLs
#Quitar emojis
#CLEAN .JSON SO IT DOES NOT SUM TWEETS IF RUNED AGAIN